In [1]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pygam import PoissonGAM, s,f, GAM, InvGaussGAM, te, l
import pickle
from scipy.stats import skellam
import itertools

# Load bike rental data with station information CSV file
bike_dat = pd.read_csv("seoul_bike_rental_data_processed_2406.csv", encoding="cp949")

# Check the shape of processed data
bike_dat.shape

(4431917, 29)

In [3]:
# For each station, find operated dates and add hours
rent_dates = bike_dat[['rent_station_number', 'rent_date']].drop_duplicates().rename(columns={'rent_station_number': 'station_number', 'rent_date': 'date'})
return_dates = bike_dat[['return_station_number', 'return_date']].drop_duplicates().rename(columns={'return_station_number': 'station_number', 'return_date': 'date'})
operated_dates = pd.concat([rent_dates, return_dates], axis=0).drop_duplicates()

hours = pd.DataFrame({"hour": np.arange(24)})
operated_hours = operated_dates.merge(hours, how="cross")

# Add station and date information
rent_station_info = bike_dat[['rent_station_number', 'rent_station_latitude', 'rent_station_longitude']].drop_duplicates().rename(
    columns={'rent_station_number': 'station_number', 'rent_station_latitude':'station_latitude', 'rent_station_longitude':'station_longitude'})
return_station_info = bike_dat[['return_station_number', 'return_station_latitude', 'return_station_longitude']].drop_duplicates().rename(
    columns={'return_station_number': 'station_number', 'return_station_latitude':'station_latitude', 'return_station_longitude':'station_longitude'})
station_info = pd.concat([rent_station_info, return_station_info], axis=0).drop_duplicates()

rent_date_info = bike_dat[['rent_date', 'rent_is_weekend', 'rent_weekday']].drop_duplicates().rename(
    columns={'rent_date': 'date', 'rent_is_weekend':'is_weekend', 'rent_weekday':'weekday'})
return_date_info = bike_dat[['return_date', 'return_is_weekend', 'return_weekday']].drop_duplicates().rename(
    columns={'return_date': 'date', 'return_is_weekend':'is_weekend', 'return_weekday':'weekday'})
date_info = pd.concat([rent_date_info, return_date_info], axis=0).drop_duplicates()

operated_hours = operated_hours.merge(station_info, how="left", on ='station_number')
operated_hours = operated_hours.merge(date_info, how="left", on ='date')

operated_hours

,station_number,date,hour,station_latitude,station_longitude,is_weekend,weekday
0,630,2024-06-01,0,37.565201,126.827316,1,5
1,630,2024-06-01,1,37.565201,126.827316,1,5
2,630,2024-06-01,2,37.565201,126.827316,1,5
3,630,2024-06-01,3,37.565201,126.827316,1,5
4,630,2024-06-01,4,37.565201,126.827316,1,5
...,...,...,...,...,...,...,...
1970347,1224,2024-07-01,19,37.518322,127.014809,0,0
1970348,1224,2024-07-01,20,37.518322,127.014809,0,0
1970349,1224,2024-07-01,21,37.518322,127.014809,0,0
1970350,1224,2024-07-01,22,37.518322,127.014809,0,0


In [5]:
# Calculate hourly bike rent and return counts
hourly_rent_counts = bike_dat.groupby(
    ['rent_station_number', 'rent_date', 'rent_hour'], as_index=False).size().rename(
    columns={'rent_station_number': 'station_number', 'rent_date': 'date', 'rent_hour':'hour', 'size': 'num_rent'})
hourly_return_counts = bike_dat.groupby(
    ['return_station_number', 'return_date', 'return_hour'], as_index=False).size().rename(
    columns={'return_station_number': 'station_number', 'return_date': 'date', 'return_hour':'hour', 'size': 'num_return'})

operated_hours = operated_hours.merge(hourly_rent_counts, how='left', on = ['station_number', 'date', 'hour']).fillna(0)
operated_hours = operated_hours.merge(hourly_return_counts, how='left', on = ['station_number', 'date', 'hour']).fillna(0)

operated_hours = operated_hours.sort_values(['station_number', 'date', 'hour'])
operated_hours = operated_hours[operated_hours['date'] != '2024-07-01']

# Add rolling average of bike rent and return counts for past 7 day-periods
operated_hours['num_rent_past_week'] = (
    operated_hours.groupby(['station_number', 'hour'])['num_rent']
    .transform(lambda x: x.rolling(7, min_periods=1).mean().shift(1)))

operated_hours['num_return_past_week'] = (
    operated_hours.groupby(['station_number', 'hour'])['num_return']
    .transform(lambda x: x.rolling(7, min_periods=1).mean().shift(1)))

operated_hours = operated_hours[pd.isna(operated_hours['num_return_past_week'])==False]

operated_hours

,station_number,date,hour,station_latitude,station_longitude,is_weekend,weekday,num_rent,num_return,num_rent_past_week,num_return_past_week
91152,0,2024-06-02,0,37.552746,126.918617,1,6,3.0,3.0,2.000000,0.000000
91153,0,2024-06-02,1,37.552746,126.918617,1,6,2.0,1.0,1.000000,1.000000
91154,0,2024-06-02,2,37.552746,126.918617,1,6,1.0,2.0,0.000000,0.000000
91155,0,2024-06-02,3,37.552746,126.918617,1,6,2.0,2.0,0.000000,0.000000
91156,0,2024-06-02,4,37.552746,126.918617,1,6,0.0,0.0,1.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...
1900939,2739,2024-06-30,19,37.534390,126.869598,1,6,13.0,18.0,10.142857,17.142857
1900940,2739,2024-06-30,20,37.534390,126.869598,1,6,4.0,13.0,7.285714,15.000000
1900941,2739,2024-06-30,21,37.534390,126.869598,1,6,5.0,19.0,8.142857,15.142857
1900942,2739,2024-06-30,22,37.534390,126.869598,1,6,6.0,18.0,5.571429,18.428571


In [7]:
# train models with first three weeks of data, test on the last week of data 
operated_hours_train = operated_hours[operated_hours['date'] < '2024-06-24'].copy()

In [9]:
# GAM with Poisson distribution of response variables, hourly bike rentals
# interactions between latitudes and longitudes of stations
# factor term of weekends or not
# smoothing terms of hours and 7-day rolling averages of bike rentals
gam_rent = PoissonGAM(te(0,1) + s(2) + s(3) + s(3, by=2) + s(4))
gam_rent.fit(operated_hours_train[['station_latitude', 'station_longitude', 'weekday', 'hour', 'num_rent_past_week']], operated_hours_train['num_rent'])

pickle.dump(gam_rent, open('gam_rent.pkl', 'wb'))
gam_rent.summary()

PoissonGAM                                                                                                
=============================================== ==========================================================
Distribution:                       PoissonDist Effective DoF:                                    118.8508
Link Function:                          LogLink Log Likelihood:                               -2351253.752
Number of Samples:                      1432680 AIC:                                          4702745.2054
                                                AICc:                                         4702745.2257
                                                UBRE:                                               3.5436
                                                Scale:                                                 1.0
                                                Pseudo R-Squared:                                   0.6311
Feature Function                  Lam

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_56800\1179328308.py:9: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  gam_rent.summary()


In [11]:
# GAM with Poisson distribution of response variables, hourly bike return
# interactions between latitudes and longitudes of stations
# factor term of weekends or not
# smoothing terms of hours and 7-day rolling averages of bike return
gam_return = PoissonGAM(te(0,1) + s(2) + s(3) + s(3, by=2) + s(4))
gam_return.fit(operated_hours_train[['station_latitude', 'station_longitude', 'weekday', 'hour', 'num_return_past_week']], operated_hours_train['num_return'])

pickle.dump(gam_return, open('gam_return.pkl', 'wb'))
gam_return.summary()

PoissonGAM                                                                                                
=============================================== ==========================================================
Distribution:                       PoissonDist Effective DoF:                                    116.7659
Link Function:                          LogLink Log Likelihood:                              -2256915.3099
Number of Samples:                      1432680 AIC:                                          4514064.1517
                                                AICc:                                         4514064.1713
                                                UBRE:                                               3.4321
                                                Scale:                                                 1.0
                                                Pseudo R-Squared:                                   0.6646
Feature Function                  Lam

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_56800\508476841.py:9: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  gam_return.summary()


In [7]:
gam_rent = pickle.load(open('gam_rent.pkl', 'rb'))
gam_return = pickle.load(open('gam_return.pkl', 'rb'))

In [9]:
# Make predictions on the test set
operated_hours_test = operated_hours[operated_hours['date'] >= '2024-06-24'].copy()

operated_hours_test['rent_pred'] = gam_rent.predict(operated_hours_test[['station_latitude', 'station_longitude', 'weekday', 'hour', 'num_rent_past_week']])
operated_hours_test['return_pred'] = gam_rent.predict(operated_hours_test[['station_latitude', 'station_longitude', 'weekday', 'hour', 'num_return_past_week']])

operated_hours_test

,station_number,date,hour,station_latitude,station_longitude,is_weekend,weekday,num_rent,num_return,num_rent_past_week,num_return_past_week,rent_pred,return_pred
1501152,0,2024-06-24,0,37.552746,126.918617,0,0,3.0,0.0,1.428571,1.428571,1.061046,1.061046
1501153,0,2024-06-24,1,37.552746,126.918617,0,0,6.0,1.0,1.571429,1.714286,0.952110,1.030104
1501154,0,2024-06-24,2,37.552746,126.918617,0,0,2.0,0.0,1.000000,1.000000,0.537175,0.537175
1501155,0,2024-06-24,3,37.552746,126.918617,0,0,0.0,0.0,0.571429,0.285714,0.321467,0.255182
1501156,0,2024-06-24,4,37.552746,126.918617,0,0,0.0,0.0,0.428571,0.285714,0.358902,0.318999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900939,2739,2024-06-30,19,37.534390,126.869598,1,6,13.0,18.0,10.142857,17.142857,7.753281,12.929856
1900940,2739,2024-06-30,20,37.534390,126.869598,1,6,4.0,13.0,7.285714,15.000000,6.058995,12.742834
1900941,2739,2024-06-30,21,37.534390,126.869598,1,6,5.0,19.0,8.142857,15.142857,6.307486,12.158075
1900942,2739,2024-06-30,22,37.534390,126.869598,1,6,6.0,18.0,5.571429,18.428571,4.320845,12.441022


In [11]:
# calculate net differences between incoming/outgoing bikes from 6AM to 6AM in next day.
operated_hours_test['hour_from_6'] = operated_hours_test['hour'].apply(lambda x: x if x >= 6 else x + 24)
operated_hours_test['date'] = pd.to_datetime(operated_hours_test['date'])
operated_hours_test['date_converted'] = operated_hours_test.apply(lambda x: x['date'] - pd.Timedelta(days=1) if x['hour_from_6'] >= 24 else x['date'], axis=1)
operated_hours_test = operated_hours_test.sort_values(by='hour_from_6')

operated_hours_test['net_rent_pred'] = operated_hours_test.groupby(['station_number', 'date_converted'])['rent_pred'].cumsum()
operated_hours_test['net_return_pred'] = operated_hours_test.groupby(['station_number', 'date_converted'])['return_pred'].cumsum()

operated_hours_test = operated_hours_test.sort_values(by=['station_number', 'date', 'hour_from_6'])

operated_hours_test

,station_number,date,hour,station_latitude,station_longitude,is_weekend,weekday,num_rent,num_return,num_rent_past_week,num_return_past_week,rent_pred,return_pred,hour_from_6,date_converted,net_rent_pred,net_return_pred
1501158,0,2024-06-24,6,37.552746,126.918617,0,0,0.0,1.0,0.285714,0.857143,1.050759,1.636545,6,2024-06-24,1.050759,1.636545
1501159,0,2024-06-24,7,37.552746,126.918617,0,0,2.0,4.0,1.142857,1.714286,2.682177,3.763566,7,2024-06-24,3.732936,5.400111
1501160,0,2024-06-24,8,37.552746,126.918617,0,0,1.0,3.0,1.714286,4.857143,3.694699,9.833710,8,2024-06-24,7.427634,15.233821
1501161,0,2024-06-24,9,37.552746,126.918617,0,0,2.0,8.0,0.714286,4.714286,1.167225,5.745173,9,2024-06-24,8.594860,20.978994
1501162,0,2024-06-24,10,37.552746,126.918617,0,0,0.0,5.0,1.714286,6.142857,1.562965,4.857579,10,2024-06-24,10.157825,25.836573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900921,2739,2024-06-30,1,37.534390,126.869598,1,6,0.0,0.0,1.142857,5.428571,1.191975,4.806026,25,2024-06-29,89.221404,106.255188
1900922,2739,2024-06-30,2,37.534390,126.869598,1,6,0.0,0.0,1.428571,2.428571,1.216840,2.007571,26,2024-06-29,90.438244,108.262758
1900923,2739,2024-06-30,3,37.534390,126.869598,1,6,0.0,0.0,0.714286,0.571429,0.584491,0.524463,27,2024-06-29,91.022735,108.787221
1900924,2739,2024-06-30,4,37.534390,126.869598,1,6,1.0,0.0,0.857143,0.857143,0.449988,0.449988,28,2024-06-29,91.472724,109.237210


In [33]:
sample = operated_hours_test[(operated_hours_test['station_number']==0)&(operated_hours_test['date_converted']=="2024-06-24")]

df = sample[['station_number', 'date', 'hour', 'rent_pred', 'net_rent_pred', 'return_pred', 'net_return_pred']].copy()
df[['rent_pred', 'net_rent_pred', 'return_pred', 'net_return_pred']] = df[['rent_pred', 'net_rent_pred', 'return_pred', 'net_return_pred']].round(2)
df['date'] = sample['date'].dt.date
markdown_table = df.to_markdown(index=False)
print(markdown_table)

|   station_number | date       |   hour |   rent_pred |   net_rent_pred |   return_pred |   net_return_pred |
|-----------------:|:-----------|-------:|------------:|----------------:|--------------:|------------------:|
|                0 | 2024-06-24 |      6 |        1.05 |            1.05 |          1.64 |              1.64 |
|                0 | 2024-06-24 |      7 |        2.68 |            3.73 |          3.76 |              5.4  |
|                0 | 2024-06-24 |      8 |        3.69 |            7.43 |          9.83 |             15.23 |
|                0 | 2024-06-24 |      9 |        1.17 |            8.59 |          5.75 |             20.98 |
|                0 | 2024-06-24 |     10 |        1.56 |           10.16 |          4.86 |             25.84 |
|                0 | 2024-06-24 |     11 |        1.83 |           11.99 |          3    |             28.84 |
|                0 | 2024-06-24 |     12 |        1.75 |           13.74 |          1.87 |             30.71 |
|

In [35]:
# calculate initial number of bikes for each station based on average bike rentals for stations
num_bike = bike_dat['bike_id'].drop_duplicates().shape[0]

initial_num_bikes = bike_dat.groupby('rent_station_number').size().reset_index(name='num_rent')

initial_num_bikes['weight'] = initial_num_bikes['num_rent'] / initial_num_bikes['num_rent'].sum()
initial_num_bikes['initial_num_bikes'] = np.floor(initial_num_bikes['weight'] * num_bike)
initial_num_bikes = initial_num_bikes.rename(columns={'rent_station_number': 'station_number'})

operated_hours_test = operated_hours_test.merge(initial_num_bikes[['station_number', 'initial_num_bikes']], how='left', on='station_number').fillna(0)

operated_hours_test

,station_number,date,hour,station_latitude,station_longitude,is_weekend,weekday,num_rent,num_return,num_rent_past_week,num_return_past_week,rent_pred,return_pred,hour_from_6,date_converted,net_rent_pred,net_return_pred,initial_num_bikes
0,0,2024-06-24,6,37.552746,126.918617,0,0,0.0,1.0,0.285714,0.857143,1.050759,1.636545,6,2024-06-24,1.050759,1.636545,18.0
1,0,2024-06-24,7,37.552746,126.918617,0,0,2.0,4.0,1.142857,1.714286,2.682177,3.763566,7,2024-06-24,3.732936,5.400111,18.0
2,0,2024-06-24,8,37.552746,126.918617,0,0,1.0,3.0,1.714286,4.857143,3.694699,9.833710,8,2024-06-24,7.427634,15.233821,18.0
3,0,2024-06-24,9,37.552746,126.918617,0,0,2.0,8.0,0.714286,4.714286,1.167225,5.745173,9,2024-06-24,8.594860,20.978994,18.0
4,0,2024-06-24,10,37.552746,126.918617,0,0,0.0,5.0,1.714286,6.142857,1.562965,4.857579,10,2024-06-24,10.157825,25.836573,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456163,2739,2024-06-30,1,37.534390,126.869598,1,6,0.0,0.0,1.142857,5.428571,1.191975,4.806026,25,2024-06-29,89.221404,106.255188,46.0
456164,2739,2024-06-30,2,37.534390,126.869598,1,6,0.0,0.0,1.428571,2.428571,1.216840,2.007571,26,2024-06-29,90.438244,108.262758,46.0
456165,2739,2024-06-30,3,37.534390,126.869598,1,6,0.0,0.0,0.714286,0.571429,0.584491,0.524463,27,2024-06-29,91.022735,108.787221,46.0
456166,2739,2024-06-30,4,37.534390,126.869598,1,6,1.0,0.0,0.857143,0.857143,0.449988,0.449988,28,2024-06-29,91.472724,109.237210,46.0


In [41]:
num_bike

39162

In [19]:
# calculate probabilties of bike shortage for each station and each hour
operated_hours_test['prob_shortage'] = operated_hours_test.apply(lambda x: skellam.cdf(-1 * x['initial_num_bikes'], x['net_return_pred'], x['net_rent_pred']), axis=1)

operated_hours_test['prob_shortage_plus_one'] = operated_hours_test.apply(lambda x: skellam.cdf(-1 * x['initial_num_bikes'] - 1, x['net_return_pred'], x['net_rent_pred']), axis=1)
operated_hours_test['prob_shortage_minus_one'] = operated_hours_test.apply(lambda x: skellam.cdf(-1 * x['initial_num_bikes'] + 1, x['net_return_pred'], x['net_rent_pred']), axis=1)

operated_hours_test['prob_shortage_reduced'] = operated_hours_test['prob_shortage_plus_one'] - operated_hours_test['prob_shortage']
operated_hours_test['prob_shortage_added'] = operated_hours_test['prob_shortage_minus_one'] - operated_hours_test['prob_shortage']

operated_hours_test

,station_number,date,hour,station_latitude,station_longitude,is_weekend,weekday,num_rent,num_return,num_rent_past_week,...,hour_from_6,date_converted,net_rent_pred,net_return_pred,initial_num_bikes,prob_shortage,prob_shortage_plus_one,prob_shortage_minus_one,prob_shortage_reduced,prob_shortage_added
0,0,2024-06-24,6,37.552746,126.918617,0,0,0.0,1.0,0.285714,...,6,2024-06-24,1.050759,1.636545,18.0,3.001727e-17,1.647877e-18,5.184311e-16,-2.836939e-17,4.884138e-16
1,0,2024-06-24,7,37.552746,126.918617,0,0,2.0,4.0,1.142857,...,7,2024-06-24,3.732936,5.400111,18.0,1.154345e-09,2.136694e-10,5.944680e-09,-9.406757e-10,4.790335e-09
2,0,2024-06-24,8,37.552746,126.918617,0,0,1.0,3.0,1.714286,...,8,2024-06-24,7.427634,15.233821,18.0,3.073772e-08,9.520906e-09,9.583554e-08,-2.121681e-08,6.509782e-08
3,0,2024-06-24,9,37.552746,126.918617,0,0,2.0,8.0,0.714286,...,9,2024-06-24,8.594860,20.978994,18.0,6.591129e-09,2.166132e-09,1.945879e-08,-4.424997e-09,1.286767e-08
4,0,2024-06-24,10,37.552746,126.918617,0,0,0.0,5.0,1.714286,...,10,2024-06-24,10.157825,25.836573,18.0,4.772446e-09,1.701273e-09,1.303902e-08,-3.071174e-09,8.266569e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456163,2739,2024-06-30,1,37.534390,126.869598,1,6,0.0,0.0,1.142857,...,25,2024-06-29,89.221404,106.255188,46.0,3.781775e-06,2.701378e-06,5.268894e-06,-1.080398e-06,1.487118e-06
456164,2739,2024-06-30,2,37.534390,126.869598,1,6,0.0,0.0,1.428571,...,26,2024-06-29,90.438244,108.262758,46.0,3.438571e-06,2.459352e-06,4.784975e-06,-9.792193e-07,1.346404e-06
456165,2739,2024-06-30,3,37.534390,126.869598,1,6,0.0,0.0,0.714286,...,27,2024-06-29,91.022735,108.787221,46.0,3.721477e-06,2.667225e-06,5.168050e-06,-1.054252e-06,1.446573e-06
456166,2739,2024-06-30,4,37.534390,126.869598,1,6,1.0,0.0,0.857143,...,28,2024-06-29,91.472724,109.237210,46.0,3.901454e-06,2.800212e-06,5.410370e-06,-1.101242e-06,1.508915e-06


In [21]:
# for all possible rides from station A to station B, given dates and hours, calculate median minutes for use and changes in probabilities of bike shortage

return_station = station_info['station_number']
rent_station = station_info['station_number']

ride_dat = pd.DataFrame(list(itertools.product(return_station, rent_station)), columns=['return_station_number', 'rent_station_number'])
ride_dat = ride_dat[ride_dat['return_station_number'] != ride_dat['rent_station_number']]

median_use_minutes = bike_dat.groupby(['rent_station_number', 'return_station_number'], as_index=False)['use_minutes'].median()
trip_counts = bike_dat.groupby(['rent_station_number', 'return_station_number'], as_index=False)['use_minutes'].size().rename(columns={'size': 'count'})

ride_dat = ride_dat.merge(median_use_minutes, how = 'left', on=['return_station_number', 'rent_station_number'])
ride_dat = ride_dat.merge(trip_counts, how = 'left', on=['return_station_number', 'rent_station_number'])
ride_dat = ride_dat[pd.isna(ride_dat['count']) == False]
#ride_dat = ride_dat[ride_dat['use_minutes']<60]

hours = pd.DataFrame({"hour": np.arange(24)})
dates = operated_hours_test['date'].drop_duplicates()
ride_dat = ride_dat.merge(dates, how="cross")
ride_dat = ride_dat.merge(hours, how="cross")

ride_dat = ride_dat.merge(station_info.rename(columns={'station_number':'rent_station_number',	'station_latitude':'rent_station_latitude',	'station_longitude':'rent_station_longitude'}), how = 'left', on = ['rent_station_number'])
ride_dat = ride_dat.merge(station_info.rename(columns={'station_number':'return_station_number', 'station_latitude':'return_station_latitude',	'station_longitude':'return_station_longitude'}), how = 'left', on = ['return_station_number'])


ride_dat

,return_station_number,rent_station_number,use_minutes,count,date,hour,rent_station_latitude,rent_station_longitude,return_station_latitude,return_station_longitude
0,630,2295,3.0,49.0,2024-06-24,0,37.568577,126.824776,37.565201,126.827316
1,630,2295,3.0,49.0,2024-06-24,1,37.568577,126.824776,37.565201,126.827316
2,630,2295,3.0,49.0,2024-06-24,2,37.568577,126.824776,37.565201,126.827316
3,630,2295,3.0,49.0,2024-06-24,3,37.568577,126.824776,37.565201,126.827316
4,630,2295,3.0,49.0,2024-06-24,4,37.568577,126.824776,37.565201,126.827316
...,...,...,...,...,...,...,...,...,...,...
63774979,657,1318,3.0,1.0,2024-06-30,19,37.571640,127.035660,37.571640,127.035660
63774980,657,1318,3.0,1.0,2024-06-30,20,37.571640,127.035660,37.571640,127.035660
63774981,657,1318,3.0,1.0,2024-06-30,21,37.571640,127.035660,37.571640,127.035660
63774982,657,1318,3.0,1.0,2024-06-30,22,37.571640,127.035660,37.571640,127.035660


In [23]:
# return station gains an additional bike while rent station loses one. 
# calculate overall changes in probabilities of bike shortage in the system accounting both stations

return_prob_shortage_reduced = operated_hours_test[['station_number', 'date', 'hour', 'prob_shortage_reduced']].rename(columns={
    'station_number':'return_station_number', 'prob_shortage_reduced': 'return_prob_shortage_reduced'
})

rent_prob_shortage_added = operated_hours_test[['station_number', 'date', 'hour', 'prob_shortage_added']].rename(columns={
    'station_number':'rent_station_number', 'prob_shortage_added': 'rent_prob_shortage_added'
})

ride_dat = ride_dat.merge(return_prob_shortage_reduced, how = 'left', on = ['return_station_number', 'date', 'hour'])
ride_dat = ride_dat.merge(rent_prob_shortage_added, how = 'left', on = ['rent_station_number', 'date', 'hour'])

ride_dat['prob_shortage_improved'] = ride_dat['return_prob_shortage_reduced'] + ride_dat['rent_prob_shortage_added']
ride_dat['prob_shortage_improved'] = ride_dat['prob_shortage_improved'] * -1
ride_dat['prob_improved_per_min'] = ride_dat['prob_shortage_improved'] / ride_dat['use_minutes']

ride_dat = ride_dat[ride_dat['prob_shortage_improved']>0.05]
ride_dat = ride_dat[ride_dat['use_minutes']<30]

ride_dat

,return_station_number,rent_station_number,use_minutes,count,date,hour,rent_station_latitude,rent_station_longitude,return_station_latitude,return_station_longitude,return_prob_shortage_reduced,rent_prob_shortage_added,prob_shortage_improved,prob_improved_per_min
121266,476,1192,19.0,2.0,2024-06-29,18,37.486736,126.947517,37.467094,126.949326,-0.063475,0.006597,0.056878,0.002994
121267,476,1192,19.0,2.0,2024-06-29,19,37.486736,126.947517,37.467094,126.949326,-0.069332,0.007623,0.061709,0.003248
121268,476,1192,19.0,2.0,2024-06-29,20,37.486736,126.947517,37.467094,126.949326,-0.080808,0.014902,0.065906,0.003469
121269,476,1192,19.0,2.0,2024-06-29,21,37.486736,126.947517,37.467094,126.949326,-0.083494,0.014312,0.069182,0.003641
121270,476,1192,19.0,2.0,2024-06-29,22,37.486736,126.947517,37.467094,126.949326,-0.087821,0.014906,0.072916,0.003838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63774427,2170,1676,8.0,1.0,2024-06-28,19,37.526386,126.902756,37.519463,126.902649,-0.093774,0.001286,0.092488,0.011561
63774428,2170,1676,8.0,1.0,2024-06-28,20,37.526386,126.902756,37.519463,126.902649,-0.090511,0.002172,0.088339,0.011042
63774429,2170,1676,8.0,1.0,2024-06-28,21,37.526386,126.902756,37.519463,126.902649,-0.087646,0.004240,0.083406,0.010426
63774430,2170,1676,8.0,1.0,2024-06-28,22,37.526386,126.902756,37.519463,126.902649,-0.085187,0.004269,0.080918,0.010115


In [25]:
ride_dat = ride_dat[ride_dat['use_minutes']<30]

ride_dat

,return_station_number,rent_station_number,use_minutes,count,date,hour,rent_station_latitude,rent_station_longitude,return_station_latitude,return_station_longitude,return_prob_shortage_reduced,rent_prob_shortage_added,prob_shortage_improved,prob_improved_per_min
121266,476,1192,19.0,2.0,2024-06-29,18,37.486736,126.947517,37.467094,126.949326,-0.063475,0.006597,0.056878,0.002994
121267,476,1192,19.0,2.0,2024-06-29,19,37.486736,126.947517,37.467094,126.949326,-0.069332,0.007623,0.061709,0.003248
121268,476,1192,19.0,2.0,2024-06-29,20,37.486736,126.947517,37.467094,126.949326,-0.080808,0.014902,0.065906,0.003469
121269,476,1192,19.0,2.0,2024-06-29,21,37.486736,126.947517,37.467094,126.949326,-0.083494,0.014312,0.069182,0.003641
121270,476,1192,19.0,2.0,2024-06-29,22,37.486736,126.947517,37.467094,126.949326,-0.087821,0.014906,0.072916,0.003838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63774427,2170,1676,8.0,1.0,2024-06-28,19,37.526386,126.902756,37.519463,126.902649,-0.093774,0.001286,0.092488,0.011561
63774428,2170,1676,8.0,1.0,2024-06-28,20,37.526386,126.902756,37.519463,126.902649,-0.090511,0.002172,0.088339,0.011042
63774429,2170,1676,8.0,1.0,2024-06-28,21,37.526386,126.902756,37.519463,126.902649,-0.087646,0.004240,0.083406,0.010426
63774430,2170,1676,8.0,1.0,2024-06-28,22,37.526386,126.902756,37.519463,126.902649,-0.085187,0.004269,0.080918,0.010115


In [27]:
#ride_dat.to_csv("ride_dat.csv", index=False)

ride_dat_0630 = ride_dat[ride_dat['date']=="2024-06-30"]
ride_dat_0630.to_csv("ride_dat_0630.csv", index=False)

In [29]:
ride_dat_streamlit = ride_dat_0630[ride_dat_0630['hour']==8]
ride_dat_streamlit = ride_dat_streamlit[['return_station_number', 'rent_station_number', 'use_minutes', 'rent_station_latitude',	'rent_station_longitude',	'return_station_latitude',	'return_station_longitude', 'prob_shortage_improved']]

ride_dat_streamlit.to_csv("ride_dat_streamlit.csv", index=False)